# Example 4 - Spatial variation of Curie depth

In Example 2 and 3 we computed the Curie depth for a single point using a fixed window. Here, compute the Curie depth over a magnetic anomaly and estimate its uncertainty.

### Contents

- [Optimisation routine](#Optimisation-routine)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pycurious

In [ ]:
# load x,y,anomaly
mag_data = np.loadtxt("../../data/test_mag_data.txt")

nx, ny = 305, 305

x = mag_data[:,0]
y = mag_data[:,1]
d = mag_data[:,2].reshape(ny,nx)

xmin, xmax = x.min(), x.max()
ymin, ymax = y.min(), y.max()

# initialise CurieOptimise object
grid = pycurious.CurieOptimiseTanaka(d, xmin, xmax, ymin, ymax)

## Optimisation routine

Here we iteratively evaluate the Curie depth across the magnetic anomaly. We use gradient-based inversion which is deterministic, because it doesn't include uncertainty, but a quick means to recover Curie depth from the magnetic anomaly.

In [ ]:
# get centroids

window_size = 200e3
xc_list, yc_list = grid.create_centroid_list(window_size, spacingX=10e3, spacingY=10e3)

print("number of centroids = {}".format(len(xc_list)))

In [ ]:
zt, z0, zt_int, z0_int, zt_stdev, z0_stdev = grid.optimise_routine(window_size, xc_list, yc_list, taper=None)

CPD, sigma_CPD = grid.calculate_CPD(zt, z0, zt_stdev, z0_stdev)

In [ ]:
# get dimensions of domain
xcoords = np.unique(xc_list)
ycoords = np.unique(yc_list)
nc, nr = xcoords.size, ycoords.size


# plot results
fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(12,4.))

im1 = ax1.imshow(CPD.reshape(nr,nc))
im2 = ax2.imshow(sigma_CPD.reshape(nr,nc))

fig.colorbar(im1, ax=ax1, label="CPD (km)")
fig.colorbar(im2, ax=ax2, label="CPD stdev (km)")

ax1.set_title('Curie depth')
ax2.set_title('Uncertainty')